In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde, iqr
from astropy.stats import knuth_bin_width, bayesian_blocks

# 定义不同组距计算方法
def calculate_bin_count(data, method='fd'):
    N = len(data)
    if method == 'sqrt':
        return int(np.sqrt(N))
    elif method == 'sturges':
        return int(np.log2(N) + 1)
    elif method == 'fd':
        bin_width = 2 * iqr(data) / np.cbrt(N)
        return max(1, int((data.max() - data.min()) / bin_width))
    elif method == 'scott':
        bin_width = 3.5 * np.std(data) / np.cbrt(N)
        return max(1, int((data.max() - data.min()) / bin_width))
    else:
        raise ValueError("Invalid method")

# 定义自适应组距计算方法
def calculate_adaptive_bins(data, method='knuth'):
    if method == 'knuth':
        bin_width = knuth_bin_width(data)
        bins = int((data.max() - data.min()) / bin_width)
    elif method == 'bayesian_blocks':
        bins = bayesian_blocks(data)
    else:
        raise ValueError("Invalid method")
    return bins

# 绘制直方图
def plot_histogram(ax, data, bins, color, label):
    counts, bin_edges, _ = ax.hist(data, bins=bins, density=True, alpha=0.6, edgecolor='black', color=color, label=label)
    return counts, bin_edges

# 绘制KDE曲线
def plot_kde(ax, data):
    kde = gaussian_kde(data)
    kde_x = np.linspace(data.min(), data.max(), 1000)
    kde_y = kde(kde_x)
    ax.plot(kde_x, kde_y, 'blue', linewidth=1, label='KDE')
    ax.fill_between(kde_x, kde_y, where=((kde_x >= data.min()) & (kde_x <= data.max())), color='blue', alpha=0.1)
    return kde_x, kde_y

# 标注直方图
def annotate_histogram(ax, bin_edges, counts, bin_probs, row_index, out0):
    reproducibility = 0
    for j in range(len(counts)):
        if bin_edges[j] <= out0.loc[row_index].item() < bin_edges[j+1]:
            reproducibility = bin_probs[j]
            print(bin_probs[j])
            break
        ax.text(bin_edges[j] + (bin_edges[j+1] - bin_edges[j]) / 2, counts[j], f'{bin_probs[j]:.2f}', fontsize=8, ha='center')
    return reproducibility

# 绘制对比图
def plot_hist_and_kde_comparison(row_data, row_index, row_name, ax, color, out0):
    bins = calculate_adaptive_bins(row_data, method="bayesian_blocks")
    counts, bin_edges = plot_histogram(ax, row_data, bins, color, f'Bins {row_name}')
    
    bin_widths = np.diff(bin_edges)
    bin_probs = counts * bin_widths
    plot_kde(ax, row_data)
    
    ax.axvline(out0.loc[row_index].item(), color='red', linestyle='dashed', linewidth=1, label='Original Result')
    reproducibility = annotate_histogram(ax, bin_edges, counts, bin_probs, row_index, out0)
    
    ax.set_xlabel('Value')
    ax.set_ylabel('Density')
    ax.set_title(f'p of {row_name} : {reproducibility:.4f}')
    return reproducibility

# 初始化子图
def initialize_subplots(data, plotrowcount):
    num_rows = (data.shape[0] + plotrowcount - 1) // plotrowcount
    fig, axs = plt.subplots(num_rows, plotrowcount, figsize=(plotrowcount * 5, num_rows * 3.2))
    axs = axs.flatten()  # 确保 axs 是一维数组，便于索引
    return fig, axs, num_rows

# 隐藏多余的子图
def hide_extra_subplots(axs, data, num_rows, plotrowcount):
    for i in range(len(data), num_rows * plotrowcount):
        fig.delaxes(axs[i])

# 创建图例
def create_legend(axs):
    handles, labels = [], []
    for ax in axs:
        for handle, label in zip(*ax.get_legend_handles_labels()):
            if label not in labels:
                handles.append(handle)
                labels.append(label)
    return handles, labels

rowName= ["MAE", "MAPE", "RMSE", "R2", "Adj.R2", "AIC"]
total_probability_sum = 0
plotrowcount = 2
colors = plt.get_cmap('tab10')

fig, axs, num_rows = initialize_subplots(data, plotrowcount)

for index, (row, name) in enumerate(zip(data.iterrows(), rowName)):
    ax = axs[index]
    total_probability_sum += plot_hist_and_kde_comparison(row[1], index, name, ax, colors(index % 10), out0)

hide_extra_subplots(axs, data, num_rows, plotrowcount)

handles, labels = create_legend(axs)
fig.legend(handles, labels, loc='center left', bbox_to_anchor=(0.85, 0.66))

plt.subplots_adjust(hspace=0.1, wspace=0.1)
average_probability = total_probability_sum / data.shape[0]
# plt.suptitle(f'The summary of reproducbility p: {average_probability*100:.4f}%', fontsize=14)
plt.tight_layout(rect=[0, 0, 0.85, 0.85])
plt.show()